In [1]:
import pandas as pd
import math
import os
import numpy as np
import matplotlib as plt
import datetime

#library for representing the graphs
import networkx as nx

#library for Bellman Ford Algorithm
import bellmanford as bf

### Verifying Price Data

In [ ]:
PATH_TO_FILES = "/home/pau/Ironhack/week9/final_project/Project-Week-8-Final-Project/your-project/code/forex-rsi-and-bbpp-multiperiod-m1h4"
filescsv = os.listdir(PATH_TO_FILES)

In [ ]:
CURRENT_PATH = os.getcwd()
os.chdir(PATH_TO_FILES)
os.getcwd()

Arrived to the conclusion that the times in the rows are not the same for each pair of currency, there is a part of data cleaning and wrangling in order to be coherent 

In [ ]:
n_nans = []
count = []
first_datetime = []

for file in range(len(filescsv)):
    df = pd.read_csv(filescsv[file])
    n_nans.append(df["Price"].isna().sum())
    count.append(df["time"].count())
    first_datetime.append(df["time"][0])
    print(f"{file}  {filescsv[file]}","\t"+"Number of NaNs:",n_nans[-1],"\t",count[-1],"\t",first_datetime[-1])

In [ ]:
files_info_df = pd.DataFrame({"file_name":filescsv,"number_nans":n_nans,"number_rows":count,"first_datetime":first_datetime})
files_info_df.head()

In [ ]:
files_info_df[files_info_df["number_rows"] == files_info_df["number_rows"].min()]

### Creating csv file including all the pairs forex

In [ ]:
df_raw = pd.read_csv(filescsv[7]) #I decided 
currency = {"time": df_raw["time"], df_raw["Symbol()"][0][:-5]: df_raw["Price"]}
currency_df = pd.DataFrame(currency)
currency_df.head()

In [ ]:
new_filescsv = filescsv[:7] + filescsv[8:]

In [ ]:
for i in range(len(new_filescsv)):
    print(f"{i}: {new_filescsv[i]}")
    df_raw = pd.read_csv(new_filescsv[i])
    df_raw.set_index("time", drop=True, inplace=True)
    df_raw = df_raw.reindex(currency_df["time"])
    name = df_raw["Symbol()"].dropna()
    df_raw.reset_index(inplace=True)
    currency_df[name[name.index[0]][:-5]] = df_raw["Price"]

In [ ]:
currency_df

In [ ]:
currency_df.to_csv("arbitrage_forex_48_pairs_currency_v2.csv")

### Creating the Pairs Table (18 currencies)

In [2]:
PATH_TO_FILES = "/home/pau/Ironhack/week9/final_project/Project-Week-8-Final-Project/your-project/code"
filescsv = os.listdir(PATH_TO_FILES)

In [3]:
CURRENT_PATH = os.getcwd()
os.chdir(PATH_TO_FILES)
os.getcwd()

'/home/pau/Ironhack/week9/final_project/Project-Week-8-Final-Project/your-project/code'

In [5]:
currency_df = pd.read_csv("arbitrage_forex_48_pairs_currency_v2.csv", index_col=0)

In [6]:
currency_df.head()

,time,USDZAR,GBPNZD,USDCAD,GBPCHF,EURJPY,GBPSGD,SGDJPY,NZDUSD,USDDKK,...,USDHUF,CHFJPY,GBPDKK,EURSGD,CADCHF,NZDJPY,USDPLN,EURSEK,EURZAR,GBPJPY
0,2015.01.02 08:06:00,11.6121,2.00382,1.16495,1.55132,145.089,2.06340,90.682,0.77537,6.17818,...,261.05,120.663,9.60547,1.59917,0.85616,NaN,3.5505,NaN,13.9950,187.211
1,2015.01.02 08:07:00,11.6096,NaN,1.16494,1.55114,145.098,2.06347,90.685,NaN,NaN,...,261.53,120.672,9.60468,1.59930,0.85614,93.392,3.5504,NaN,13.9935,NaN
2,2015.01.02 08:08:00,11.6071,2.00319,1.16487,1.55102,145.104,2.06328,90.686,0.77568,6.17735,...,261.02,120.672,9.60282,1.59940,0.85614,93.415,NaN,NaN,13.9945,187.203
3,2015.01.02 08:09:00,11.6071,2.00368,1.16489,1.55076,145.110,2.06333,90.677,0.77560,6.17645,...,261.53,120.689,9.60204,1.59960,0.85597,93.390,3.5496,NaN,13.9965,187.201
4,2015.01.02 08:10:30,11.6096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.43678,NaN,NaN


In [7]:
list_headers = currency_df.columns.tolist()
list_headers = list_headers[1:]
list_currencies = []
for position in list_headers:
    list_currencies.append(position[:3])
    list_currencies.append(position[3:])
set_currencies = set(list_currencies)
unique_currencies = sorted(list(set_currencies))

In [8]:
pairs_table_df = pd.DataFrame(0, index=unique_currencies, columns=unique_currencies)

In [9]:
d = {currency: [0] for currency in unique_currencies}
pairs_table_df = pd.DataFrame(data=d, index=unique_currencies)
pairs_table_df = pairs_table_df.sort_index()
pairs_table_df.columns = pairs_table_df.index

In [10]:
def deleting_rows_columns_only_nans(df):
    columns_to_remove = []
    for column in df.columns:
        dropping = df[column].drop(column)
        if dropping.isna().all():
            columns_to_remove.append(column)
    df.drop(columns_to_remove, axis=1, inplace=True)
    df.drop(columns_to_remove, inplace=True)
    return df

In [12]:
SAVING_PATH = "/home/pau/Ironhack/week9/final_project/Project-Week-8-Final-Project/your-project/code/possibles_arbitrages_currencies"

for index in range(len(currency_df["time"][0:])): #I wrote "20" in this case because I wanted to iterate only the first 20 tables, to iterate until the end I have to write "[0:]"
    for row in pairs_table_df.columns:
        for column in pairs_table_df.index:
            try:
                if row == column:
                    pairs_table_df.loc[row,column] = 1
                else:
                    pairs_table_df.loc[row,column] = currency_df.loc[index][1:][row+column]
                    pairs_table_df.loc[column,row] = 1/currency_df.loc[index][1:][row+column]    # perquè és "df.iloc" i no "df.loc" ????
            except:
                if pairs_table_df.loc[row,column] == 0:
                    pairs_table_df.loc[row,column] = np.NaN
    print(f"{index}   Iteration {currency_df.time[index]}")
    pairs_table_df = deleting_rows_columns_only_nans(pairs_table_df)
    pairs_table_df.to_csv(f"{SAVING_PATH}/{currency_df.time[index]}_possible_arbitrage.csv")
    pairs_table_df = pd.DataFrame(0, index=unique_currencies, columns=unique_currencies)

0   Iteration 2015.01.02 08:06:00
1   Iteration 2015.01.02 08:07:00
2   Iteration 2015.01.02 08:08:00
3   Iteration 2015.01.02 08:09:00
4   Iteration 2015.01.02 08:10:30
5   Iteration 2015.01.02 08:11:00
6   Iteration 2015.01.02 08:12:30
7   Iteration 2015.01.02 08:13:00
8   Iteration 2015.01.02 08:14:00
9   Iteration 2015.01.02 08:15:00
10   Iteration 2015.01.02 08:16:00
11   Iteration 2015.01.02 08:17:00
12   Iteration 2015.01.02 08:18:00
13   Iteration 2015.01.02 08:19:00
14   Iteration 2015.01.02 08:20:00
15   Iteration 2015.01.02 08:21:00
16   Iteration 2015.01.02 08:22:00
17   Iteration 2015.01.02 08:23:00
18   Iteration 2015.01.02 08:24:00
19   Iteration 2015.01.02 08:25:00
20   Iteration 2015.01.02 08:26:30
21   Iteration 2015.01.02 08:27:00
22   Iteration 2015.01.02 08:28:00
23   Iteration 2015.01.02 08:29:00
24   Iteration 2015.01.02 08:30:00
25   Iteration 2015.01.02 08:31:00
26   Iteration 2015.01.02 08:32:00
27   Iteration 2015.01.02 08:33:00
28   Iteration 2015.01.02 08:3

231   Iteration 2015.01.02 11:59:00
232   Iteration 2015.01.02 12:00:00
233   Iteration 2015.01.02 12:01:00
234   Iteration 2015.01.02 12:02:00
235   Iteration 2015.01.02 12:03:00
236   Iteration 2015.01.02 12:04:00
237   Iteration 2015.01.02 12:05:01
238   Iteration 2015.01.02 12:06:00
239   Iteration 2015.01.02 12:07:00
240   Iteration 2015.01.02 12:08:00
241   Iteration 2015.01.02 12:09:00
242   Iteration 2015.01.02 12:10:00
243   Iteration 2015.01.02 12:11:00
244   Iteration 2015.01.02 12:12:00
245   Iteration 2015.01.02 12:13:00
246   Iteration 2015.01.02 12:14:00
247   Iteration 2015.01.02 12:15:00
248   Iteration 2015.01.02 12:16:00
249   Iteration 2015.01.02 12:17:00
250   Iteration 2015.01.02 12:18:00
251   Iteration 2015.01.02 12:19:00
252   Iteration 2015.01.02 12:20:01
253   Iteration 2015.01.02 12:21:00
254   Iteration 2015.01.02 12:22:00
255   Iteration 2015.01.02 12:23:00
256   Iteration 2015.01.02 12:24:00
257   Iteration 2015.01.02 12:25:00
258   Iteration 2015.01.02 1

KeyboardInterrupt: 

### Arbitrage opportunities

In [24]:
PATH_TO_FILES = "/home/pau/Ironhack/week9/final_project/Project-Week-8-Final-Project/your-project/code/possibles_arbitrages_currencies"
filescsv = os.listdir(PATH_TO_FILES)

In [25]:
CURRENT_PATH = os.getcwd()
os.chdir(PATH_TO_FILES)
os.getcwd()

'/home/pau/Ironhack/week9/final_project/Project-Week-8-Final-Project/your-project/code/possibles_arbitrages_currencies'

### Bellman Ford Algorithm to find the arbitrage opportunity

In [15]:
def create_graph(currencies):
    G = nx.DiGraph()
    for series in currencies:
        G.add_nodes_from(series.index)
        # """series.index[i] = ROWS""" """series.name = COLUMNS"""
        G.add_edges_from([(series.index[i], series.name, {'weight':-np.log(series[i])}) for i in range(len(series))])
    return G

In [16]:
def columns_formatter(df):
    series = []
    for i in df.columns:
        values = df[i].apply(lambda x: x if x !=1.0 else np.nan)
        values = values.dropna()
        series.append(values)
    return series 

In [17]:
def calculation_profit(clist):
    total = 1
    for i in range(1,len(clist)):
        total *= arb_df.loc[clist[i-1]][clist[i]]
    return total

In [18]:
def print_profit_results(results,unique_paths):
    currencies_paths = results[1]
    profit = calculation_profit(currencies_paths)
    if (profit > 1.00005) and (currencies_paths not in unique_paths):
        print(f"Path: {currencies_paths}")
        for i in range(1,len(currencies_paths)):
            print(f"{currencies_paths[i-1]}-{currencies_paths[i]}",arb_df.loc[currencies_paths[i-1]][currencies_paths[i]]) #arb_df.loc[ROWS][COLUMNS]
        print(f"Profit: {profit-1}")
        print("\n")
        unique_paths.append(currencies_paths)
    return unique_paths

In [26]:
timestamp = []
path_id = []
complete_path = []
source = []
target = []
change = []
profit_lst = []

path_counter = 0
for file in range(len(filescsv)):
#for file in range(0,8):
    print(f"{file}   ############################## {filescsv[file]} ##############################")
    arb_df = pd.read_csv(filescsv[file], index_col=0)
    currencies = columns_formatter(arb_df)
    G = create_graph(currencies)
    unique_paths = []
    for i in arb_df.index:
        results = bf.bellman_ford(G, source=i, target=i, weight="weight")
        currencies_paths = results[1]
        profit = calculation_profit(currencies_paths)
        if (profit > 1.00005) and (currencies_paths not in unique_paths):
            #print(f"Path: {currencies_paths}")
            for j in range(1,len(currencies_paths)):
                timestamp.append(filescsv[file])
                source.append(currencies_paths[j-1])
                target.append(currencies_paths[j])
                change.append(arb_df.loc[currencies_paths[j-1]][currencies_paths[j]])
                #print(f"{source[-1]}-{target[-1]}",change[-1]) #arb_df.loc[ROWS][COLUMNS]
                path_id.append(path_counter)
                complete_path.append(currencies_paths)
                profit_lst.append(profit-1)
            path_counter += 1
            #print(f"Profit: {profit-1}")
            #print("\n")
            unique_paths.append(currencies_paths)

0   ############################## 2015.01.02 10:35:00_possible_arbitrage.csv ##############################
1   ############################## 2015.01.02 09:41:00_possible_arbitrage.csv ##############################
2   ############################## 2015.01.02 10:15:00_possible_arbitrage.csv ##############################
3   ############################## 2015.01.02 09:18:00_possible_arbitrage.csv ##############################
4   ############################## 2015.01.02 13:26:00_possible_arbitrage.csv ##############################
5   ############################## 2015.01.02 13:28:00_possible_arbitrage.csv ##############################
6   ############################## 2015.01.02 10:51:00_possible_arbitrage.csv ##############################
7   ############################## 2015.01.02 13:46:00_possible_arbitrage.csv ##############################
8   ############################## 2015.01.02 12:41:00_possible_arbitrage.csv ##############################
9   ###############

76   ############################## 2015.01.02 11:02:00_possible_arbitrage.csv ##############################
77   ############################## 2015.01.02 11:59:00_possible_arbitrage.csv ##############################
78   ############################## 2015.01.02 08:08:00_possible_arbitrage.csv ##############################
79   ############################## 2015.01.02 12:28:00_possible_arbitrage.csv ##############################
80   ############################## 2015.01.02 10:04:00_possible_arbitrage.csv ##############################
81   ############################## 2015.01.02 11:18:00_possible_arbitrage.csv ##############################
82   ############################## 2015.01.02 09:21:00_possible_arbitrage.csv ##############################
83   ############################## 2015.01.02 13:44:00_possible_arbitrage.csv ##############################
84   ############################## 2015.01.02 12:03:00_possible_arbitrage.csv ##############################
85   #####

154   ############################## 2015.01.02 10:10:00_possible_arbitrage.csv ##############################
155   ############################## 2015.01.02 13:07:00_possible_arbitrage.csv ##############################
156   ############################## 2015.01.02 10:56:00_possible_arbitrage.csv ##############################
157   ############################## 2015.01.02 09:51:00_possible_arbitrage.csv ##############################
158   ############################## 2015.01.02 11:33:00_possible_arbitrage.csv ##############################
159   ############################## 2015.01.02 12:16:00_possible_arbitrage.csv ##############################
160   ############################## 2015.01.02 10:30:00_possible_arbitrage.csv ##############################
161   ############################## 2015.01.02 14:25:00_possible_arbitrage.csv ##############################
162   ############################## 2015.01.02 09:20:00_possible_arbitrage.csv ##############################
1

235   ############################## 2015.01.02 10:29:00_possible_arbitrage.csv ##############################
236   ############################## 2015.01.02 08:39:00_possible_arbitrage.csv ##############################
237   ############################## 2015.01.02 11:08:00_possible_arbitrage.csv ##############################
238   ############################## 2015.01.02 14:08:00_possible_arbitrage.csv ##############################
239   ############################## 2015.01.02 11:05:00_possible_arbitrage.csv ##############################
240   ############################## 2015.01.02 08:48:00_possible_arbitrage.csv ##############################
241   ############################## 2015.01.02 13:25:00_possible_arbitrage.csv ##############################
242   ############################## 2015.01.02 08:37:00_possible_arbitrage.csv ##############################
243   ############################## 2015.01.02 13:02:00_possible_arbitrage.csv ##############################
2

314   ############################## 2015.01.02 10:34:00_possible_arbitrage.csv ##############################
315   ############################## 2015.01.02 11:04:00_possible_arbitrage.csv ##############################
316   ############################## 2015.01.02 08:49:00_possible_arbitrage.csv ##############################
317   ############################## 2015.01.02 11:55:00_possible_arbitrage.csv ##############################
318   ############################## 2015.01.02 10:14:00_possible_arbitrage.csv ##############################
319   ############################## 2015.01.02 10:07:00_possible_arbitrage.csv ##############################
320   ############################## 2015.01.02 13:22:00_possible_arbitrage.csv ##############################
321   ############################## 2015.01.02 10:31:00_possible_arbitrage.csv ##############################
322   ############################## 2015.01.02 08:14:00_possible_arbitrage.csv ##############################
3

In [27]:
x = pd.DataFrame({"timestamp":timestamp,"path_id":path_id,"complete_path":complete_path,"source":source,"target":target,"change":change,"profit":profit_lst})
print(len(timestamp))
print(len(path_id))
print(len(complete_path))
print(len(source))
print(len(target))
print(len(change))
print(len(profit_lst))

8768
8768
8768
8768
8768
8768
8768


In [28]:
x

,timestamp,path_id,complete_path,source,target,change,profit
0,2015.01.02 10:35:00_possible_arbitrage.csv,0,"[EUR, NOK, GBP, EUR]",EUR,NOK,9.045600,0.000103
1,2015.01.02 10:35:00_possible_arbitrage.csv,0,"[EUR, NOK, GBP, EUR]",NOK,GBP,0.085845,0.000103
2,2015.01.02 10:35:00_possible_arbitrage.csv,0,"[EUR, NOK, GBP, EUR]",GBP,EUR,1.287930,0.000103
3,2015.01.02 10:35:00_possible_arbitrage.csv,1,"[CAD, AUD, JPY, CAD]",CAD,AUD,1.054363,0.000181
4,2015.01.02 10:35:00_possible_arbitrage.csv,1,"[CAD, AUD, JPY, CAD]",AUD,JPY,98.020000,0.000181
5,2015.01.02 10:35:00_possible_arbitrage.csv,1,"[CAD, AUD, JPY, CAD]",JPY,CAD,0.009678,0.000181
6,2015.01.02 10:35:00_possible_arbitrage.csv,2,"[NZD, AUD, GBP, NOK, USD, NZD]",NZD,AUD,0.953262,0.000580
7,2015.01.02 10:35:00_possible_arbitrage.csv,2,"[NZD, AUD, GBP, NOK, USD, NZD]",AUD,GBP,0.524541,0.000580
8,2015.01.02 10:35:00_possible_arbitrage.csv,2,"[NZD, AUD, GBP, NOK, USD, NZD]",GBP,NOK,11.648900,0.000580
9,2015.01.02 10:35:00_possible_arbitrage.csv,2,"[NZD, AUD, GBP, NOK, USD, NZD]",NOK,USD,0.133270,0.000580
